In [15]:
import jsons
import numpy as np
import pandas as pd
from treelib import Tree as TreelibTree

import ipywidgets as widgets
from IPython.display import display, clear_output

In [16]:
class Node:
    def __init__(self, value=None, left=None, right=None, depth=0):
        self.value = value
        self.left = left
        self.right = right
        self.depth = depth

    def is_leaf(self):
        return self.right is None and self.left is None

    @staticmethod
    def from_dict(data_dict, depth=0): # Renomeado 'dict' para 'data_dict' para evitar sombrear o tipo built-in
        if data_dict is None:
            return None

        node = Node(data_dict['value'], depth=depth)
        node.right = Node.from_dict(data_dict['right'], depth=depth + 1)
        node.left = Node.from_dict(data_dict['left'], depth=depth + 1)
        return node

In [17]:
class BinaryDecisionTreeClassifier:
    def __init__(self, filename=None):
        self.root = None
        if filename is not None:
            self.load(filename)

    def fit(self, X, y):
        self.root = self._build_tree(X, y)

    def predict(self, X):
        return np.array([self._traverse_tree(X.iloc[i], self.root) for i in range(len(X))])

    def print_tree(self):
        if self.root is None:
            return

        treelib_tree_instance = TreelibTree()  # Use a classe renomeada
        BinaryDecisionTreeClassifier.create_treelib_tree(treelib_tree_instance, self.root)
        treelib_tree_instance.show()

    @staticmethod
    def create_treelib_tree(tree_instance, node_custom, parent_custom=None): 
        if parent_custom is None:
            # Use o valor do nó como rótulo e id(node_custom) como identificador único
            tree_instance.create_node(tag=node_custom.value, identifier=id(node_custom)) 
        else:
            # Use o valor do nó como rótulo e id(node_custom) como identificador único
            # O nó pai é identificado por id(parent_custom)
            tree_instance.create_node(tag=node_custom.value, identifier=id(node_custom), parent=id(parent_custom)) 

        # Recursivamente cria os nós filhos
        if node_custom.left is not None:
            BinaryDecisionTreeClassifier.create_treelib_tree(tree_instance, node_custom.left, node_custom)
        if node_custom.right is not None:
            BinaryDecisionTreeClassifier.create_treelib_tree(tree_instance, node_custom.right, node_custom)

    def _drop_features_with_same_values(self, X):
        for feature in X.columns:
            if len(set(X[feature])) == 1:
                X = X.drop(columns=feature)
        return X

    def _build_tree(self, X, y, depth=0):
        X = self._drop_features_with_same_values(X)
        n_samples, n_features = X.shape

        if n_features == 0 and len(y) > 0:
            return BinaryDecisionTreeClassifier.create_subtree_from_names_list(y, depth=depth)

        if n_features == 0 and len(y) == 1: 
            return Node(y[0], depth=depth)

        if len(y) == 0:
            return Node() 

        if n_samples == 1 and len(y) == 1: 
            return Node(y[0], depth=depth)

        best_feature = self._choose_split_feature(X)
        if best_feature is None: 
             return BinaryDecisionTreeClassifier.create_subtree_from_names_list(y, depth=depth)


        X_left, y_left, X_right, y_right = self._split_data(X, y, best_feature)

        left_subtree = self._build_tree(X_left, y_left, depth + 1)
        right_subtree = self._build_tree(X_right, y_right, depth + 1)

        return Node(best_feature, left_subtree, right_subtree, depth=depth)

    @staticmethod
    def create_subtree_from_names_list(names_list, depth=0):
        if not names_list.any(): # Lida com array numpy vazio
            return Node()

        if len(names_list) == 1:
            return Node(names_list[0], depth=depth)

        # Estratégia de divisão simples para folhas quando não há mais features
        # Pode ser melhorada, por exemplo, pegando o nome mais comum se houver duplicatas
        # ou criando uma cadeia de perguntas "É X?", "É Y?"
        # A implementação original parece pegar o primeiro e o resto.
        current_name = names_list[0]
        remaining_names = names_list[1:]

        # Se não houver nomes restantes, o nó esquerdo será uma folha vazia ou com um nome padrão
        if not remaining_names.any():
            left_subtree = Node("Desconhecido_Esquerda", depth=depth + 1) # Ou alguma outra lógica
        else:
            left_subtree = BinaryDecisionTreeClassifier.create_subtree_from_names_list(
                remaining_names, depth=depth+1)

        right_subtree = Node(current_name, depth=depth+1) # Nó direito é o nome atual

        return Node(f"É {current_name}?", left_subtree, right_subtree, depth=depth)


    def _choose_split_feature(self, X):
        best_feature = None
        min_absolute_sum = float('inf') # Renomeado de best_entropy para refletir o cálculo

        if X.empty: # Adicionado para checar se X está vazio
            return None

        for feature in X.columns:
            values = X[feature].values
            # A heurística original é 'abs(sum(values))'.
            # Uma entropia mais tradicional ou ganho de informação seria mais comum.
            current_absolute_sum = abs(sum(values))

            if current_absolute_sum < min_absolute_sum:
                best_feature = feature
                min_absolute_sum = current_absolute_sum
            elif current_absolute_sum == min_absolute_sum:
                if best_feature is None or (feature < best_feature):
                    best_feature = feature

        return best_feature


    def _split_data(self, X, y, best_feature):
        left_idx, = np.where(X[best_feature] == -1)
        right_idx, = np.where(X[best_feature] == 1)

        X_processed = X.drop(columns=best_feature)

        X_left = X_processed.iloc[left_idx]
        X_right = X_processed.iloc[right_idx]

        y_left = y[left_idx]
        y_right = y[right_idx]

        return X_left, y_left, X_right, y_right

    def _traverse_tree(self, x_row, node): # x_row é uma linha (Series) de X
        if node is None or node.is_leaf(): # Adicionado node is None check
            return node.value if node else None

        feature_name = node.value
        if feature_name not in x_row:
            return self._traverse_tree(x_row, node.left)


        if x_row[feature_name] == -1:
            return self._traverse_tree(x_row, node.left)
        else:
            return self._traverse_tree(x_row, node.right)

    def load(self, filename='tree.json'):
        with open(filename, 'r') as f:
            data_dict = jsons.loads(f.read()) 
        self.root = Node.from_dict(data_dict['root'])

    def dump(self, filename='tree.json'):
        json_str = jsons.dumps({"root": self.root}, strip_nulls=True) 
        with open(filename, 'w') as f:
            f.write(str(json_str))

    def drop(self):
        self.root = None

    @staticmethod
    def get_max_depth_from_node(node):
        if node is None or node.value is None: 
            return 0
        if node.is_leaf():
            return 1
        
        # Garante que left e right não são None antes de chamar recursivamente
        left_depth = BinaryDecisionTreeClassifier.get_max_depth_from_node(node.left) if node.left else 0
        right_depth = BinaryDecisionTreeClassifier.get_max_depth_from_node(node.right) if node.right else 0
        
        return 1 + max(left_depth, right_depth)


    @staticmethod
    def get_deepest_subtree(node):
        if node is None or node.is_leaf(): # Adicionado node is None check
            return ('leaf', node) if node else ('leaf', None)

        left_depth = BinaryDecisionTreeClassifier.get_max_depth_from_node(node.left)
        right_depth = BinaryDecisionTreeClassifier.get_max_depth_from_node(node.right)

        if left_depth > right_depth:
            return 'left', node.left
        elif right_depth > left_depth:
            return 'right', node.right
        else:
            # Desempate: arbitrariamente escolher esquerda, ou poderia ser baseado em outro critério
            return 'left', node.left

In [18]:
class Akinator:
    id_counter = 0 

    def __init__(self, tree_classifier): 
        if not tree_classifier.root:
            raise ValueError("Tree has not been trained yet")

        self.id = Akinator.id_counter
        Akinator.id_counter += 1

        self.tree_classifier = tree_classifier
        self._current_node = tree_classifier.root
        self.not_sure_nodes_stack = [] 

    def answer_question(self, answer_value): 
        if answer_value not in [-1, 1, 0]: # Não, Sim, Talvez/Não Sei
            raise ValueError("Answer must be -1, 1 or 0")

        if self._current_node is None or self._current_node.is_leaf():
            return self.current_question 

        if answer_value == 1: # Sim
            self._current_node = self._current_node.right
        elif answer_value == -1: # Não
            self._current_node = self._current_node.left
        else: # Talvez/Não Sei (0)
            # Lógica para "não sei": pega o galho mais profundo
            direction, deepest_node_in_branch = BinaryDecisionTreeClassifier.get_deepest_subtree(self._current_node)
            # Guarda o nó atual e a direção que *não* foi tomada para poder voltar atrás
            if direction == 'left': # Se o mais profundo foi para a esquerda
                self.not_sure_nodes_stack.append(('right', self._current_node)) # Guardamos o caminho da direita
            else: # Se o mais profundo foi para a direita (ou folha)
                self.not_sure_nodes_stack.append(('left', self._current_node)) # Guardamos o caminho da esquerda
            self._current_node = deepest_node_in_branch


        # Se o novo _current_node for None
        if self._current_node is None:
            # Isso pode indicar um problema na árvore ou na lógica
            pass


        return self.current_question


    def add_person(self, new_name, new_feature_question):
        if self._current_node is None or not self._current_node.is_leaf():
            # Idealmente, chamar add_person apenas quando uma folha (suposição errada) é alcançada.
            print("Atenção: add_person deve ser chamado quando o Akinator faz uma suposição final (nó folha).")
            return

        # O valor do nó folha atual é a pessoa que o Akinator achou que era.
        old_person_name = self._current_node.value
        current_depth = self._current_node.depth

        # Transforma o nó folha atual em um nó de pergunta
        self._current_node.value = new_feature_question # Ex: "É famoso por cantar?"
        
        # O filho da direita é a nova pessoa (que responde SIM à nova pergunta)
        self._current_node.right = Node(new_name, depth=current_depth + 1)
        # O filho da esquerda é a pessoa original (que responde NÃO à nova pergunta)
        self._current_node.left = Node(old_person_name, depth=current_depth + 1)

        self.tree_classifier.dump() # Salva a árvore modificada


    def get_progress(self):
        if self._current_node is None or self.tree_classifier.root is None:
            return 1.0 # Ou 0.0, dependendo de como quer representar progresso inválido

        # Profundidade total da subárvore a partir do nó atual
        # (get_max_depth_from_node retorna 1 para folha, então subtrai 1 para "níveis restantes")
        remaining_depth = BinaryDecisionTreeClassifier.get_max_depth_from_node(self._current_node)
        if remaining_depth > 0: remaining_depth -=1


        # Profundidade total da árvore inteira a partir da raiz
        total_tree_depth = BinaryDecisionTreeClassifier.get_max_depth_from_node(self.tree_classifier.root)
        if total_tree_depth > 0: total_tree_depth -=1
        
        if total_tree_depth == 0: # Árvore é apenas a raiz
            return 1.0 if self._current_node.is_leaf() else 0.0

        current_node_depth_from_root = self._current_node.depth
        
        # O progresso pode ser a profundidade atual / profundidade total da árvore
        # Ou (profundidade_total - profundidade_restante_na_subarvore) / profundidade_total
        # A segunda parece mais intuitiva para o "quanto já foi perguntado"
        
        # Evitar divisão por zero se a árvore for vazia ou só a raiz (sem profundidade de perguntas)
        if total_tree_depth == 0:
            return 1.0 # Considera 100% se não há perguntas a fazer

        progress = current_node_depth_from_root / total_tree_depth
        return min(round(progress, 2), 1.0) # Garante que não passe de 1.0 devido a arredondamentos ou lógica


    def continue_game(self):
        if not self.not_sure_nodes_stack:
            # Não há mais caminhos alternativos a tentar
            return {"question": self._current_node.value if self._current_node else None, "done": True, "progress": 1.0}

        if self._current_node is None or not self._current_node.is_leaf():
            # Se não estamos numa folha, significa que o "não sei" anterior não nos levou a uma.
            # Isso pode acontecer se a árvore for muito rasa ou get_deepest_subtree não encontrar uma folha.
            # Neste caso, vamos tentar o próximo caminho alternativo.
            pass # Prossegue para pegar da pilha

        # Pega o último nó onde respondemos "não sei" e tenta o outro caminho
        direction_to_take, fallback_node = self.not_sure_nodes_stack.pop()

        if direction_to_take == 'left':
            self._current_node = fallback_node.left
        else: # 'right'
            self._current_node = fallback_node.right
        
        # Se o novo caminho for inválido (None), o que não deveria ocorrer em árvore bem formada
        if self._current_node is None:
             return {"question": None, "done": True, "progress": 1.0, "error": "Invalid path in continue_game"}


        return self.current_question


    @property
    def current_question(self):
        if self._current_node:
            done = self._current_node.is_leaf()
            question_or_guess = self._current_node.value
            progress = self.get_progress()
            return {"question": question_or_guess, "done": done, "progress": progress}
        return {"question": None, "done": True, "progress": 1.0, "error": "Current node is None"}

In [19]:
try:
    df = pd.read_csv('dataset.csv')
    X = df[df.columns[1:]] # Características
    y = df['nome'].values    # Rótulos (nomes das personalidades)
    print("Dataset carregado com sucesso.")
    print(f"X shape: {X.shape}, y shape: {y.shape}")
except FileNotFoundError:
    print("Erro: dataset.csv não encontrado. Crie um ou ajuste o caminho.")
    X, y = None, None # Define para None para evitar erros subsequentes

# Opção 1: Treinar uma nova árvore a partir do CSV
if X is not None and y is not None:
    print("Treinando a árvore de decisão...")
    tree = BinaryDecisionTreeClassifier()
    tree.fit(X, y)
    print("Árvore treinada. Visualizando:")
    tree.print_tree()
    # Opcional: Salvar a árvore treinada
    # tree.dump('tree.json')
    # print("Árvore salva em tree.json")
else:
    print("Não foi possível treinar a árvore pois o dataset não foi carregado.")
    tree = None

# RUN Akinator
if tree and tree.root:
    akinator_instance = Akinator(tree)
    print("\nInstância do Akinator criada.")
    print(f"Primeira pergunta: {akinator_instance.current_question}")
else:
    print("Akinator não pôde ser inicializado pois a árvore está vazia ou não foi carregada/treinada.")
    akinator_instance = None
    
if 'akinator_instance' in globals() and akinator_instance is not None:
    # --- Widgets ---
    question_label = widgets.HTML(value="Carregando...")
    progress_bar = widgets.FloatProgress(value=0.0, min=0.0, max=1.0, description='Progresso:', bar_style='info')

    btn_sim = widgets.Button(description="Sim", button_style='success', icon='check')
    btn_nao = widgets.Button(description="Não", button_style='danger', icon='times')
    btn_talvez = widgets.Button(description="Talvez/Não Sei", button_style='warning', icon='question')

    btn_acertei = widgets.Button(description="Sim, Acertou!", button_style='success', icon='smile-o')
    btn_errei = widgets.Button(description="Não, Errou!", button_style='danger', icon='frown-o')

    btn_ensinar_sim = widgets.Button(description="Sim, quero ensinar", button_style='primary', icon='plus')
    btn_ensinar_nao = widgets.Button(description="Não, obrigado", button_style='info', icon='thumbs-down')

    text_new_person_name = widgets.Text(value='', placeholder='Nome da pessoa/personagem')
    text_new_feature = widgets.Text(value='', placeholder='Uma pergunta que o diferencie (Ex: É cantor?)')
    btn_save_new_person = widgets.Button(description="Salvar Novo Personagem", button_style='primary', icon='save')
    
    feedback_label = widgets.HTML(value="") # Para mensagens como "Obrigado por ensinar!"

    # Output widget para controlar a exibição e evitar flickering
    output_area = widgets.Output()
    
    # Layouts
    answer_buttons = widgets.HBox([btn_sim, btn_nao, btn_talvez])
    guess_confirm_buttons = widgets.HBox([btn_acertei, btn_errei])
    teach_prompt_buttons = widgets.HBox([btn_ensinar_sim, btn_ensinar_nao])
    new_person_form = widgets.VBox([
        widgets.HTML(value="<b>Por favor, me ensine:</b>"),
        text_new_person_name, 
        text_new_feature, 
        btn_save_new_person
    ])

    # --- Estado do Jogo (para controlar visibilidade dos widgets) ---
    game_stage = "asking" # "asking", "guessing", "teaching_prompt", "teaching_form", "finished"

    # --- Funções Handler ---
    def update_display():
        with output_area:
            clear_output(wait=True) # Limpa a área de output antes de redesenhar
            
            current_q_data = akinator_instance.current_question
            if current_q_data.get("error"):
                question_label.value = f"<p style='color:red;'>Erro: {current_q_data['error']}</p>"
                display(question_label)
                return

            progress_bar.value = current_q_data['progress']
            
            display(progress_bar)
            question_label.value = f"<h3>{current_q_data['question']}</h3>"
            display(question_label)
            display(feedback_label) # Exibe feedback se houver

            if game_stage == "asking":
                feedback_label.value = "" # Limpa feedback anterior
                display(answer_buttons)
            elif game_stage == "guessing":
                display(guess_confirm_buttons)
            elif game_stage == "teaching_prompt":
                display(teach_prompt_buttons)
            elif game_stage == "teaching_form":
                # A pergunta original (que foi o palpite errado) precisa ser visível para o usuário formular a nova característica.
                # `akinator_instance.current_question` ainda terá o palpite errado neste ponto.
                question_label.value = (f"<h3>Você pensou em alguém que não era '{current_q_data['question']}'.</h3>"
                                        f"<p>Forneça um nome e uma pergunta que seja <b>VERDADEIRA</b> para sua escolha e <b>FALSA</b> para '{current_q_data['question']}'.</p>")

                display(new_person_form)
            elif game_stage == "finished":
                # O feedback_label já terá a mensagem final.
                pass


    def handle_answer(answer_value):
        global game_stage
        akinator_instance.answer_question(answer_value)
        current_q_data = akinator_instance.current_question
        
        if current_q_data.get("error"):
            game_stage = "finished" # Ou um estágio de erro
        elif current_q_data['done']:
            game_stage = "guessing"
        else:
            game_stage = "asking"
        update_display()

    def on_btn_sim_clicked(b): handle_answer(1)
    def on_btn_nao_clicked(b): handle_answer(-1)
    def on_btn_talvez_clicked(b): handle_answer(0)

    btn_sim.on_click(on_btn_sim_clicked)
    btn_nao.on_click(on_btn_nao_clicked)
    btn_talvez.on_click(on_btn_talvez_clicked)

    def on_btn_acertei_clicked(b):
        global game_stage
        feedback_label.value = "<h3 style='color:green;'>Eba! Acertei! :)</h3><p>Obrigado por jogar!</p>"
        game_stage = "finished"
        update_display()

    def on_btn_errei_clicked(b):
        global game_stage
        feedback_label.value = "<p>Hmm, que pena. Deixe-me tentar alternativas...</p>"
        akinator_instance.continue_game() # Tenta encontrar outro caminho
        
        continued_state = akinator_instance.current_question # Pega o estado após o continue_game
        
        if continued_state.get("error"):
            feedback_label.value = f"<p style='color:red;'>Erro ao tentar continuar: {continued_state['error']}</p>"
            game_stage = "finished"
        elif not continued_state['done']: # Se encontrou uma nova pergunta
            game_stage = "asking"
        else: # Se ainda está 'done' (sem mais alternativas ou novo palpite final)
            feedback_label.value = (f"<p>Parece que não tenho mais alternativas ou meu novo palpite ('{continued_state['question']}') também não é o correto.</p>"
                                   "<p>Gostaria de me ensinar quem era?</p>")
            game_stage = "teaching_prompt"
        update_display()

    btn_acertei.on_click(on_btn_acertei_clicked)
    btn_errei.on_click(on_btn_errei_clicked)

    def on_btn_ensinar_sim_clicked(b):
        global game_stage
        game_stage = "teaching_form"
        feedback_label.value = "" # Limpa feedback anterior
        update_display()
        
    def on_btn_ensinar_nao_clicked(b):
        global game_stage
        feedback_label.value = "<h3>Ok, obrigado por jogar!</h3>"
        game_stage = "finished"
        update_display()

    btn_ensinar_sim.on_click(on_btn_ensinar_sim_clicked)
    btn_ensinar_nao.on_click(on_btn_ensinar_nao_clicked)

    def on_btn_save_new_person_clicked(b):
        global game_stage
        new_name = text_new_person_name.value.strip()
        new_feature_q = text_new_feature.value.strip()

        if not new_name or not new_feature_q:
            feedback_label.value = "<p style='color:red;'>Por favor, preencha o nome e a pergunta característica.</p>"
            update_display() # Para mostrar o feedback sem mudar o estágio
            return

        # Adiciona a pessoa. A lógica em akinator.add_person usará o _current_node (que é o palpite errado)
        # para criar os novos galhos.
        akinator_instance.add_person(new_name, new_feature_q)
        feedback_label.value = f"<h3 style='color:green;'>Obrigado! Aprendi sobre {new_name}.</h3><p>A árvore foi atualizada e salva (se a função dump estiver ativa em add_person).</p>"
        text_new_person_name.value = "" # Limpa campos
        text_new_feature.value = ""
        game_stage = "finished"
        update_display()

    btn_save_new_person.on_click(on_btn_save_new_person_clicked)

    # --- Inicialização do Display ---
    # Define o estado inicial e exibe
    initial_q_data = akinator_instance.current_question
    if initial_q_data.get("error"):
        question_label.value = f"<p style='color:red;'>Erro na inicialização: {initial_q_data['error']}</p>"
        display(question_label)
    elif initial_q_data['done']: # Caso a árvore seja apenas uma folha
        game_stage = "guessing"
    else:
        game_stage = "asking"
    
    # Exibe a área de output que conterá todos os outros widgets dinâmicos
    display(output_area)
    update_display()

else:
    print("Akinator não foi inicializado corretamente. Não é possível iniciar o jogo com widgets.")
    # pass

Dataset carregado com sucesso.
X shape: (824, 685), y shape: (824,)
Treinando a árvore de decisão...
Árvore treinada. Visualizando:
Nasceu entre 1900 e 1950?
├── Nasceu no século 19?
│   ├── Nasceu entre 1950 e 2000?
│   │   ├── Nasceu no século 18?
│   │   │   ├── Nasceu no século 17?
│   │   │   │   ├── Nasceu no século 16?
│   │   │   │   │   ├── É filósofo?
│   │   │   │   │   │   ├── Nasceu no século 15?
│   │   │   │   │   │   │   ├── É brasileiro?
│   │   │   │   │   │   │   │   ├── Nasceu no século 21?
│   │   │   │   │   │   │   │   │   ├── É atacante?
│   │   │   │   │   │   │   │   │   │   ├── Vinícius Júnior
│   │   │   │   │   │   │   │   │   │   └── É ator?
│   │   │   │   │   │   │   │   │   │       ├── Erasmo Carlos
│   │   │   │   │   │   │   │   │   │       └── Luiz Carlos Miele
│   │   │   │   │   │   │   │   │   └── É escritor?
│   │   │   │   │   │   │   │   │       ├── É advogado?
│   │   │   │   │   │   │   │   │       │   ├── Bento Teixeira
│   │   │   │   │   │

Output()